In [1]:
import pandas as pd
from sqlalchemy import create_engine
import datetime
import os

In [2]:
def save_world_to_database(df_world):
    connection_string = "postgres:postgres@localhost:5432/corona_db"
    engine = create_engine(f'postgresql://{connection_string}')
    connection = engine.connect()
    connection.execute( '''TRUNCATE TABLE daily_stats_world''' )
    connection.close()
    df_world.to_sql(name='daily_stats_world', con=engine, if_exists='append', index=False)

In [3]:
# Start here to process the world data (only call save_world_to_database)
url_world = "static/world_data/df_world_all.csv"
df_world = pd.read_csv(url_world)
df_world = df_world.drop(['country_2'], axis=1)
df_world.head()
save_world_to_database(df_world)

In [4]:
# query_str = open('static/sql/test_world.sql')
query_str = open('static/sql/world_top_10.sql')
query_text = ""
for text in query_str:
    query_text = query_text + text

connection_string = "postgres:postgres@localhost:5432/corona_db"
engine = create_engine(f'postgresql://{connection_string}')    
print(query_text)
df_query = pd.read_sql_query(query_text, con=engine)

df_query


select  
	country, 
	sum(conf_count) total,
	sum(conf_count)-sum(cured_count)-sum(dead_count) sick, 
	sum(cured_count) cured, 
	sum(dead_count) dead,
	max(date)
    from daily_stats_world 
    where date = (select max(date ) from daily_stats_world) 
    group by country 
    order by total desc 
	limit 10


,country,total,sick,cured,dead,max
0,China,79826,34838,42118,2870,2020-03-01
1,South Korea,3736,3689,30,17,2020-03-01
2,Italy,1694,1577,83,34,2020-03-01
3,Iran,978,749,175,54,2020-03-01
4,Others,705,689,10,6,2020-03-01
5,Japan,256,218,32,6,2020-03-01
6,France,130,116,12,2,2020-03-01
7,Germany,130,114,16,0,2020-03-01
8,Singapore,106,34,72,0,2020-03-01
9,Hong Kong,96,58,36,2,2020-03-01
